# Project Euler
## Problems 51 - 60

### ************************

## Problem #51

Find the smallest prime which, by replacing part of the number (not necessarily adjacent digits) with the same digit, is part of an eight prime value family.

In [41]:
import math
from itertools import combinations

In [73]:
def findEightReplacementPrime():
    primes = generatePrimesTo(5000)
    curNum = 56003
    memo = set()
    while curNum < 2100000:
        if curNum in memo or curNum % 5 == 0:
            curNum += 2
            continue
        if isPrime(curNum, primes) and canReplaceDigits(curNum, primes, memo):
            return curNum
        curNum += 2
    return None

In [71]:
def canReplaceDigits(prime, primes, memo):
    digits = numToArray(prime)
    countArr = [0 for _ in range(10)]
    for d in digits:
        countArr[d] += 1
    vals = []
    for i, x in enumerate(countArr):
        if x >= 3:
            vals.append(i)
    if len(vals) == 0:
        return False
    for val in vals:
        indeces = []
        for i, d in enumerate(digits):
            if d == val:
                indeces.append(i)
        combs = list(combinations(indeces, 3))
        d_copy = digits.copy()
        for c in combs:
            digits = d_copy
            notPrimeCount = 0
            for i in range(10):
                if i == 0 and c[0] == 0:
                    notPrimeCount += 1
                    continue
                for idx in c:
                    digits[idx] = i
                curNum = arrayToNum(digits)
                if isPrime(curNum, primes):
                    memo.add(curNum)
                else:
                    notPrimeCount += 1
                    if notPrimeCount > 2:
                        break
            if notPrimeCount < 3:
                return True
    return False

In [1]:
def numToArray(num):
    digits = []
    while num > 0:
        num, rem = divmod(num, 10)
        digits.append(rem)
    return digits[::-1]

In [4]:
def arrayToNum(arr):
    num = arr[0]
    for i in range(1, len(arr)):
        num *= 10
        num += arr[i]
    return num

In [9]:
def generatePrimesTo(n):
    primes = [2,3,5,7]
    i = 11
    while i < n:
        for j in [0, 2, 6, 8]:
            if isPrime(i+j, primes):
                primes.append(i+j)
        i += 10
    while primes[-1] > n:
        primes.pop()
    return primes

In [10]:
def isPrime(n, primes):
    if n < 11:
        return False if n not in [2,3,5,7] else True
    i = 0
    root = math.floor(n**0.5)
    while i < len(primes) and primes[i] <= root:
        if n % primes[i] == 0:
            return False
        i += 1
    return True

In [74]:
findEightReplacementPrime()

121313

## Problem #52

Find the smallest positive integer x, such that 2x, 3x, ..., 6x contain the same digits.

In [77]:
def findSmallestSameDigits():
    x = 9
    isGood = False
    while not isGood:
        counts = digitCounts(x)
        isGood = True
        for i in range(2, 7):
            val = i * x
            if counts != digitCounts(val):
                isGood = False
                x += 9
                break
    return x

In [76]:
def digitCounts(n):
    counts = [0 for _ in range(10)]
    while n > 0:
        n, rem = divmod(n, 10)
        counts[rem] += 1
    return counts

In [78]:
findSmallestSameDigits()

142857

## Problem #53

How many, not necessarily distinct, values of n choose r for 1 <= n <= 100 are greater than one-million?

In [80]:
def findGreaterThan(cap):
    count = 0
    for n in range(1, 101):
        for r in range(1, n+1):
            if choose(n, r) > cap:
                count += 1
    return count

In [79]:
def choose(n, k):
    prod = 1
    for i in range(1, k+1):
        prod = (prod * (n-i+1)) // i
    return prod

In [81]:
findGreaterThan(1000000)

4075

## Problem #54

How many poker hands does Player 1 win?

In [137]:
def pokerResults():
    hands = getHandList('poker_p54.txt')
    p1wins, p2wins = 0, 0
    pdic = pokerDictMaker()
    for hand in hands:
        p1hand = hand[:5]
        p2hand = hand[5:]
        p1score, p2score = rankOfHand(p1hand, pdic), rankOfHand(p2hand, pdic)
        if p1score > p2score:
            p1wins += 1
        elif p2score > p1score:
            p2wins += 1
        else:
            whoWins = breakPokerTie(p1hand, p2hand, p1score, pdic)
            if whoWins == 1:
                p1wins += 1
            else:
                p2wins += 1
    print("P1 Wins:", p1wins)
    print("P2 Wins:", p2wins)
    return p1wins

In [132]:
def breakPokerTie(h1, h2, rank, pdic):
    h1vals, h2vals = [], []
    for i in range(5):
        h1vals.append(pdic[h1[i][0]])
        h2vals.append(pdic[h2[i][0]])
    if rank in [1, 5, 6, 9]:
        return 1 if max(h1vals) > max(h2vals) else 2
    counts1, counts2 = {}, {}
    for i in range(5):
        val1, val2 = h1vals[i], h2vals[i]
        if val1 in counts1:
            counts1[val1] += 1
        else:
            counts1[val1] = 1
        if val2 in counts2:
            counts2[val2] += 1
        else:
            counts2[val2] = 1
    counts1, counts2 = invertDict(counts1), invertDict(counts2)
    if rank == 8:
        if counts1[4] == counts2[4]:
            return 1 if counts1[1][0] > counts2[1][0] else 2
        return 1 if counts1[4][0] > counts2[4][0] else 2
    if rank == 4 or rank == 7:
        if counts1[3] == counts2[3]:
            if 2 in counts1:
                return 1 if counts1[2][0] > counts2[2][0] else 2
            return 1 if max(counts1[1]) > max(counts1[1]) else 2
        return 1 if counts1[3][0] > counts2[3][0] else 2
    if rank == 2:
        if counts1[2] == counts2[2]:
            return 1 if max(counts1[1]) > max(counts2[1]) else 2
        return 1 if counts1[2][0] > counts2[2][0] else 2
    if max(counts1[2]) == max(counts2[2]):
        if min(counts1[2]) == min(counts2[2]):
            return 1 if counts1[1][0] > counts2[1][0] else 2
        return 1 if min(counts1[2]) > min(counts2[2]) else 2
    return 1 if max(counts1[2]) > max(counts2[2]) else 2

In [120]:
def invertDict(dct):
    newDict = {}
    for key, val in dct.items():
        if val in newDict:
            newDict[val].append(key)
        else:
            newDict[val] = [key]
    return newDict

In [121]:
def rankOfHand(hand, pdic):
    values = []
    suits = []
    for card in hand:
        values.append(pdic[card[0]])
        suits.append(card[1])
    values.sort()
    numSuits = len(set(suits))
    isFlush = numSuits == 1
    isStraight = True
    for i in range(1, len(values)):
        if values[i] != values[i-1] + 1:
            isStraight = False
            break
    if isStraight and isFlush:
        if values == [10, 11, 12, 13, 14]:
            return 10
        else:
            return 9
    if isFlush:
        return 6
    if isStraight:
        return 5
    valueDict = {}
    for val in values:
        if val in valueDict:
            valueDict[val] += 1
        else:
            valueDict[val] = 1
    counts = sorted(valueDict.values())
    if counts == [1, 4]:
        return 8
    if counts == [2, 3]:
        return 7
    if counts[-1] == 3:
        return 4
    if counts == [1,2,2]:
        return 3
    if counts[-1] == 2:
        return 2
    return 1

In [122]:
def getHandList(filename):
    hands = []
    with open(filename, 'r') as f:
        for line in f.readlines():
            line = line.strip()
            row = line.split()
            hands.append(row)
    return hands

In [123]:
def pokerDictMaker():
    values = {'T': 10, 'J': 11, 'Q':12, 'K': 13, 'A':14}
    for i in range(2, 10):
        values[str(i)] = i
    return values

In [138]:
pokerResults()

P1 Wins: 376
P2 Wins: 624


376

## Problem #55

How many Lychrel numbers are there below ten-thousand?

In [162]:
def lychrelsBelow(cap):
    ldic = {}
    count = 0
    for i in range(1, cap):
        if isLychrel(i, ldic, cap):
            count += 1
    return count

In [161]:
def isLychrel(n, lychrelDict, cap):
    if n in lychrelDict:
        return lychrelDict[n]
    i = 0
    valsChecked = [n]
    curVal = n
    while i < 50:
        curVal = curVal + arrToNum(numToArrReversed(curVal))
        if isPalindrome(numToArrReversed(curVal)):
            break
        if curVal < cap:
            valsChecked.append(curVal)
        i += 1
    if i < 50:
        for val in valsChecked:
            lychrelDict[val] = False
        return False
    else:
        for val in valsChecked:
            lychrelDict[val] = True
        return True

In [151]:
def isPalindrome(digits):
    li, ri = 0, len(digits)-1
    while li <= ri:
        if digits[li] != digits[ri]:
            return False
        li += 1
        ri -= 1
    return True

In [152]:
def numToArrReversed(n):
    digits = []
    while n > 0:
        n, rem = divmod(n, 10)
        digits.append(rem)
    return digits

In [153]:
def arrToNum(arr):
    val = arr[0]
    for i in range(1, len(arr)):
        val *= 10
        val += arr[i]
    return val

In [163]:
lychrelsBelow(10000)

249

## Problem #56

Considering natural numbers of the form a^b where a,b < 100 what is the maximum digital sum?

In [164]:
def maxDigitalSum(cap):
    maxSum = 0
    for a in range(1, cap):
        for b in range(1, cap):
            curSum = digitalSum(a**b)
            if curSum > maxSum:
                maxSum = curSum
    return maxSum

In [165]:
def digitalSum(n):
    total = 0
    while n > 0:
        n, rem = divmod(n, 10)
        total += rem
    return total

In [166]:
maxDigitalSum(100)

972

## Problem #57

In the first one-thousand expansions of sqrt(2), how many fractions contain a numerator with more digits than the denominator?

In [168]:
import math

In [174]:
def squareRootOfTwo(cap):
    n, d = 3, 2
    count = 0
    expansions = 1
    while expansions <= cap:
        n, d = n + 2*d, n + d
        if numDigits(n) != numDigits(d):
            count += 1
        expansions += 1
    return count

In [170]:
def numDigits(n):
    return math.floor(math.log10(n)) + 1

In [175]:
squareRootOfTwo(1000)

153

## Problem #58

If one complete new layer is wrapped around the spiral above, a square spiral with side length 9 will be formed. If this process is continued, what is the side length of the square spiral for which the ratio of primes along both diagonals first falls below 10%?

In [184]:
def spiralPrimePercentage(cap):
    primes = generatePrimesTo(50000)
    primeCount = 3
    compCount = 2
    i = 2
    while primeCount / (primeCount + compCount) >= cap:
        compCount += 1
        sq = (2*i+1)**2
        nums = [sq-2*i, sq-4*i, sq-6*i]
        for num in nums:
            if isPrime(num, primes):
                primeCount += 1
            else:
                compCount += 1
        i += 1
    return int(sq**0.5)

In [176]:
def generatePrimesTo(n):
    primes = [2,3,5,7]
    i = 11
    while i < n:
        for j in [0, 2, 6, 8]:
            if isPrime(i+j, primes):
                primes.append(i+j)
        i += 10
    while primes[-1] > n:
        primes.pop()
    return primes

In [177]:
def isPrime(n, primes):
    if n < 11:
        return False if n not in [2,3,5,7] else True
    i = 0
    root = math.floor(n**0.5)
    while i < len(primes) and primes[i] <= root:
        if n % primes[i] == 0:
            return False
        i += 1
    return True

In [185]:
spiralPrimePercentage(0.1)

26241

## Problem #59

The encryption key consists of three lower case characters. Using a file containing the encrypted ASCII codes, and the knowledge that the plain text must contain common English words, decrypt the message and find the sum of the ASCII values in the original text.

In [315]:
from itertools import product

In [338]:
def asciiSum():
    message = runXorDecryption()
    total = 0
    for char in message:
        total += ord(char)
    print(message)
    return total

In [334]:
def runXorDecryption():
    nums = getListFromFile('xorDecryption_p59.txt')
    alpha = 'abcdefghijklmnopqrstuvwxyz'
    commonWords = set()
    targetMessage = ""
    with open('commonWords_p59.txt', 'r') as f:
        for line in f.readlines():
            line = line.strip()
            commonWords.add(line)
    for code in product(alpha, repeat=3):
        i = 0
        newMessage = []
        while i < len(nums):
            codeVal = ord(code[i % 3])
            msgVal = nums[i]
            char = chr(msgVal ^ codeVal)
            newMessage.append(char)
            i += 1
        message = ''.join(newMessage).split()
        counts = 0
        for word in message:
            if word in commonWords:
                counts += 1
        if counts > 20:
            return ' '.join(message)

In [206]:
def getListFromFile(filename):
    with open(filename, 'r') as f:
        message = ''.join(f.readlines()).split(',')
    return [int(x) for x in message]

In [207]:
def intListToText(nums):
    message = [chr(x) for x in nums]
    return ''.join(message)

In [339]:
asciiSum()

An extract taken from the introduction of one of Euler's most celebrated papers, "De summis serierum reciprocarum" [On the sums of series of reciprocals]: I have recently found, quite unexpectedly, an elegant expression for the entire sum of this series 1 + 1/4 + 1/9 + 1/16 + etc., which depends on the quadrature of the circle, so that if the true sum of this series is obtained, from it at once the quadrature of the circle follows. Namely, I have found that the sum of this series is a sixth part of the square of the perimeter of the circle whose diameter is 1; or by putting the sum of this series equal to s, it has the ratio sqrt(6) multiplied by s to 1 of the perimeter to the diameter. I will soon show that the sum of this series to be approximately 1.644934066842264364; and from multiplying this number by six, and then taking the square root, the number 3.141592653589793238 is indeed produced, which expresses the perimeter of a circle whose diameter is 1. Following again the same ste

129448

## Problem #60

Find the lowest sum for a set of five primes for which any two primes concatenate to produce another prime.

In [422]:
import math

In [434]:
def primePairSets(target):
    primes = generatePrimesTo(20000)
    validSets = []
    for p in primes:
        memo = set()
        for group in validSets:
            works = True
            for val in group:
                if not canConcat(p, val, primes, memo):
                    works = False
                    break
            if works:
                if len(group) + 1 == target:
                    finalList = sorted(list(group) + [p])
                    print(finalList)
                    return sum(finalList)
                validSets.append(set([p] + list(group)))
        validSets.append(set([p]))

In [435]:
def canConcat(p, q, primes, memo):
    if q in memo:
        return True
    result = isPrime(concat(p, q), primes) and isPrime(concat(q, p), primes)
    if result:
        memo.add(q)
    return result

In [436]:
def concat(a, b):
    numTens = math.floor(math.log10(b)) + 1
    return a * 10**numTens + b

In [437]:
def generatePrimesTo(n):
    primes = [2,3,5,7]
    i = 11
    while i < n:
        for j in [0, 2, 6, 8]:
            if isPrime(i+j, primes):
                primes.append(i+j)
        i += 10
    while primes[-1] > n:
        primes.pop()
    return primes

In [438]:
def isPrime(n, primes):
    if n < 11:
        return False if n not in [2,3,5,7] else True
    i = 0
    root = math.floor(n**0.5)
    while i < len(primes) and primes[i] <= root:
        if n % primes[i] == 0:
            return False
        i += 1
    return True

In [439]:
primePairSets(5)

[13, 5197, 5701, 6733, 8389]


26033